In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os

# --- EDIT YOUR FILE PATHS HERE ---
ENROLMENT_PATH = '/content/drive/MyDrive/UDAI/Clean_DataSet/structured_Unified_DataSet/Aadhar_enrolment_Str.csv'
BIOMETRIC_PATH = '/content/drive/MyDrive/UDAI/Clean_DataSet/structured_Unified_DataSet/Aadhar_biometric_Str.csv'
DEMOGRAPHIC_PATH = '/content/drive/MyDrive/UDAI/Clean_DataSet/structured_Unified_DataSet/Aadhar_demographic_Str.csv'

# Helper function to clean and load data
def load_and_clean(path):
    if not os.path.exists(path):
        print(f"Error: File not found at {path}")
        return None
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    # Clean string data to ensure matches
    for col in ['state', 'district']:
        if col in df.columns:
            df[col] = df[col].str.strip().str.lower()
    return df

print("Setup Complete. Ready to load data.")

Setup Complete. Ready to load data.


In [7]:
df_enrol = load_and_clean(ENROLMENT_PATH)
df_bio = load_and_clean(BIOMETRIC_PATH)
df_demo = load_and_clean(DEMOGRAPHIC_PATH)

if df_enrol is not None and df_bio is not None:
    enrol_dists = set(df_enrol['district'].unique())
    bio_dists = set(df_bio['district'].unique())
    common = enrol_dists.intersection(bio_dists)

    print(f"Audit Results:")
    print(f"- Districts in Enrolment: {len(enrol_dists)}")
    print(f"- Districts in Biometric: {len(bio_dists)}")
    print(f"- Common Districts Found: {len(common)}")

    if len(common) == 0:
        print("!!! WARNING: No common districts. Gap Analysis will be empty. Check file names.")
    else:
        print("SUCCESS: Geographic overlap confirmed.")

Audit Results:
- Districts in Enrolment: 916
- Districts in Biometric: 900
- Common Districts Found: 886
SUCCESS: Geographic overlap confirmed.


In [8]:
# 1. Aggregate Demographic Updates (Low Tech)
d_total = df_demo.groupby(['state', 'district'])[['demo_age_5_17', 'demo_age_17_']].sum().sum(axis=1).reset_index()
d_total.columns = ['state', 'district', 'total_demo']

# 2. Aggregate Biometric Updates (High Tech)
b_total = df_bio.groupby(['state', 'district'])[['bio_age_5_17', 'bio_age_17_']].sum().sum(axis=1).reset_index()
b_total.columns = ['state', 'district', 'total_bio']

# 3. Merge and Calculate Infrastructure Ratio
# bio_to_demo_ratio: How many biometrics are scanned per 100 demographic changes
gap2_df = pd.merge(d_total, b_total, on=['state', 'district'], how='inner')
gap2_df['infra_ratio_pct'] = (gap2_df['total_bio'] / (gap2_df['total_demo'] + 1)) * 100

# 4. Define Policy Categories
def infra_status(ratio):
    if ratio < 5: return 'Critical: Hardware Shortage'
    if ratio < 20: return 'Warning: Equipment Lag'
    return 'Balanced: Full Service'

gap2_df['infra_status'] = gap2_df['infra_ratio_pct'].apply(infra_status)

print("Gap Analysis #2 Calculation Complete.")
print(gap2_df[['state', 'district', 'infra_ratio_pct', 'infra_status']].head())

Gap Analysis #2 Calculation Complete.
                       state                  district  infra_ratio_pct  \
0  andaman & nicobar islands                  andamans       268.308921   
1  andaman & nicobar islands                   nicobar       229.822335   
2  andaman & nicobar islands                  nicobars        40.000000   
3  andaman & nicobar islands  north and middle andaman       317.562189   
4  andaman & nicobar islands             south andaman       283.662429   

             infra_status  
0  Balanced: Full Service  
1  Balanced: Full Service  
2  Balanced: Full Service  
3  Balanced: Full Service  
4  Balanced: Full Service  


In [9]:
# Aggregate to State Level
state_infra = gap2_df.groupby('state').agg({
    'total_demo': 'sum',
    'total_bio': 'sum'
}).reset_index()

state_infra['state_ratio'] = (state_infra['total_bio'] / (state_infra['total_demo'] + 1)) * 100

fig_state_gap2 = px.bar(
    state_infra.sort_values('state_ratio'),
    x='state_ratio',
    y='state',
    color='state_ratio',
    color_continuous_scale='RdYlGn', # Red to Green
    orientation='h',
    title='<b>State-Level Hardware Audit: Biometric Service Availability</b>',
    labels={'state_ratio': 'Biometric Update Capacity (%)'},
    template='plotly_white'
)

# Add target benchmark line at 20%
fig_state_gap2.add_vline(x=20.0, line_dash="dash", line_color="black", annotation_text="Healthy Infrastructure Floor")

fig_state_gap2.show()

In [10]:
# Sort by the most critical districts (lowest ratio)
fig_dist_gap2 = px.bar(
    gap2_df.sort_values('infra_ratio_pct').head(20),
    x='infra_ratio_pct',
    y='district',
    color='infra_status',
    orientation='h',
    text_auto='.1f',
    color_discrete_map={
        'Critical: Hardware Shortage': '#8b0000',
        'Warning: Equipment Lag': '#f39c12',
        'Balanced: Full Service': '#27ae60'
    },
    title='<b>Top 20 Districts: Hardware Access Gap</b><br><sup>Lower % indicates citizens lack access to Biometric Scanners</sup>',
    labels={'infra_ratio_pct': 'Biometric Capacity (%)', 'district': 'District'}
)

fig_dist_gap2.update_layout(template='plotly_white', height=600)
fig_dist_gap2.show()

In [11]:
fig_scatter = px.scatter(
    gap2_df,
    x='total_demo',
    y='total_bio',
    size='infra_ratio_pct',
    color='infra_status',
    hover_name='district',
    hover_data=['state'],
    title='<b>The Identity Friction Map: Demographic Demand vs. Biometric Capacity</b>',
    labels={'total_demo': 'Demographic Volume (Demand)', 'total_bio': 'Biometric Volume (Capacity)'},
    color_discrete_map={
        'Critical: Hardware Shortage': '#8b0000',
        'Warning: Equipment Lag': '#f39c12',
        'Balanced: Full Service': '#27ae60'
    }
)

# Add a 45-degree line to show 1:1 parity
max_val = max(gap2_df['total_demo'].max(), gap2_df['total_bio'].max())
fig_scatter.add_shape(type="line", x0=0, y0=0, x1=max_val, y1=max_val, line=dict(color="gray", dash="dot"))

fig_scatter.update_layout(template='plotly_white', height=600)
fig_scatter.show()

In [12]:
# --- RE-RUN GAP ANALYSIS #2 AND CAPTURE REPORT ---

def get_infra_report(demo_df, bio_df):
    # 1. Aggregate Demographic Updates
    d_total = demo_df.groupby(['state', 'district'])[['demo_age_5_17', 'demo_age_17_']].sum().sum(axis=1).reset_index()
    d_total.columns = ['state', 'district', 'total_demo']

    # 2. Aggregate Biometric Updates
    b_total = bio_df.groupby(['state', 'district'])[['bio_age_5_17', 'bio_age_17_']].sum().sum(axis=1).reset_index()
    b_total.columns = ['state', 'district', 'total_bio']

    # 3. Merge and Calculate Ratio
    report = pd.merge(d_total, b_total, on=['state', 'district'], how='inner')
    report['bio_to_demo_ratio'] = (report['total_bio'] / (report['total_demo'] + 1)) * 100
    return report

# Generate the report (ensure df_demo and df_bio are defined from your previous cells)
infra_report = get_infra_report(df_demo, df_bio)

# PRINT THE TOP 50 HARDWARE SHORTAGE DISTRICTS
print("--- DATA FOR GAP ANALYSIS #2 (HARDWARE) ---")
top_infra_gap = infra_report.sort_values('bio_to_demo_ratio').head(10)
print(f"Top Hardware Shortage Districts (Lowest Ratio First):")
for i, row in top_infra_gap.iterrows():
    print(f"- {row['district'].title()} ({row['state'].title()}): Ratio {row['bio_to_demo_ratio']:.2f}%")

--- DATA FOR GAP ANALYSIS #2 (HARDWARE) ---
Top Hardware Shortage Districts (Lowest Ratio First):
- Salumbar (Rajasthan): Ratio 0.47%
- Phalodi (Rajasthan): Ratio 0.98%
- Didwanakuchaman (Rajasthan): Ratio 1.11%
- Pherzawl (Manipur): Ratio 1.13%
- Balotra (Rajasthan): Ratio 1.19%
- Kotputlibehror (Rajasthan): Ratio 1.32%
- Khairthaltijara (Rajasthan): Ratio 1.42%
- Beawar (Rajasthan): Ratio 1.57%
- Namchi (Sikkim): Ratio 1.77%
- Jalore (Rajasthan): Ratio 1.93%


In [13]:
# Final Export for Gap 2
# Using the correct variable 'gap2_df' and column 'infra_ratio_pct' found in your notebook
gap2_export = gap2_df[['state', 'district', 'total_demo', 'total_bio', 'infra_ratio_pct']].copy()

# Standardize for merging
gap2_export['state'] = gap2_export['state'].str.strip().str.lower()
gap2_export['district'] = gap2_export['district'].str.strip().str.lower()

gap2_export.to_csv('/content/drive/MyDrive/UDAI/Collab /Results/gap2_results.csv', index=False)
print("Gap 2 Exported successfully as gap2_results.csv")

Gap 2 Exported successfully as gap2_results.csv


# 🔍 Data-Driven Insights: Gap Analysis #2: Service Infrastructure Audit

This audit evaluates the **Technical Accessibility** of Aadhaar services by comparing the volume of demographic updates (low-tech) against biometric updates (high-tech). A low ratio indicates that while citizens are trying to update their IDs, the local centers lack functional **Iris or Fingerprint scanners**.

---

### 🛑 1. The "Maroon Zone": Critical Hardware Shortages
Our analysis identifies a severe infrastructure bottleneck in the **Rajasthan cluster**, where the ratio of biometric capacity is nearly non-existent.

* **The Evidence:** * **Salumbar (Rajasthan):** 0.47% ratio
    * **Phalodi (Rajasthan):** 0.98% ratio
    * **Didwana Kuchaman (Rajasthan):** 1.11% ratio
    * **Balotra (Rajasthan):** 1.19% ratio
* **The Verdict:** In these districts, for every 100 people changing their address or phone number, **fewer than 2 people** are able to update their biometrics. This is a clear signal of **Hardware Depletion**—centers are likely turning citizens away because scanners are broken or missing.

### ⚠️ 2. The "Operational Lag" in the North East & Central India
While not as critical as the Maroon Zone, several districts in **Assam** and **Chhattisgarh** show a significant lag in equipment availability.

* **The Evidence:** * **Bajali (Assam):** 2.72% ratio
    * **Sribhumi (Assam):** 7.02% ratio
    * **Manendragarh (Chhattisgarh):** 11.60% ratio
* **The Verdict:** These areas are in a **"Partial Service Gap."** Citizens have limited access to biometric services, likely leading to long queues and travel times to neighboring districts for simple biometric refreshes.

### 📈 3. Emerging Service Stability
We see signs of balanced infrastructure as we move into West Bengal and Telangana.

* **The Evidence:** **Vikarabad (Telangana)** at 23.49% and **North Dinajpur (West Bengal)** at 24.38%.
* **The Verdict:** While still not at 100%, these districts represent the "Healthy Baseline" for 2025. They have enough functional scanners to handle roughly 1 biometric update for every 4 demographic updates.

---

## 🛠️ Infrastructure Priority Action Plan

| Priority Level | Data Signal (Ratio) | Targeted Districts | Recommended Policy Action |
| :--- | :--- | :--- | :--- |
| **🚨 CRITICAL** | **< 2%** | **Salumbar, Phalodi, Balotra** | **Emergency Deployment:** Ship 50+ Biometric Scanner Kits immediately. |
| **⚠️ WARNING** | **2% - 15%** | **Bajali, Dima Hasao, Deeg** | **Hardware Audit:** Send technical teams to repair/replace existing scanners. |
| **🟡 MONITOR** | **15% - 30%** | **Koch Bihar, Bhabua, Mauganj** | **Capacity Expansion:** Hire more operators to utilize existing hardware. |
| **✅ STABLE** | **> 30%** | **West Medinipur, Raigad** | **Strategic Reserve:** Use these districts as hubs for mobile update vans. |

---
